In [0]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16 

In [0]:
#Connect to Goole Mount
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#for using GPU, Install kmc2
!pip install kmc2

In [0]:
from sklearn.model_selection import train_test_split
from imutils import paths
import numpy as np
import imutils 
import cv2 
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import copy
import pickle
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn import preprocessing
#from libKMCUDA import kmeans_cuda # kmeans-gpu 사용 --> not installed

In [0]:
파일처리 해주기 
Label과 Data  매칭시키기

In [0]:
df_data=pd.read_csv('/content/drive/My Drive/2019-ml-finalproject/Label2Names.csv',header=None)

DATA_ROOT_TRAIN="/content/drive/My Drive/2019-ml-finalproject/train"
train_des=list()
train_labels=list()

for cls in tqdm(os.listdir(DATA_ROOT_TRAIN)): #진행도를 알려줌
  img_list = os.listdir(DATA_ROOT_TRAIN+'/'+cls)
  
  if cls == 'BACKGROUND_Google' :
    label= 101
  else :
    label=(df_data.index[df_data[1]==cls]+1).tolist()[0]
  
  for img in img_list:  
    image=cv2.imread(DATA_ROOT_TRAIN+'/'+cls+'/'+img)
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    train_des.append(gray)
    train_labels.append(label)




In [0]:
#피클사용 저장
#with open('/content/drive/My Drive/2019-ml-finalproject/train_labels.pickle','wb') as f:
#  pickle.dump(train_labels,f,pickle.HIGHEST_PROTOCOL)
#with open('/content/drive/My Drive/2019-ml-finalproject/train_des.pickle','wb') as f:
  pickle.dump(train_des,f,pickle.HIGHEST_PROTOCOL)
#print(np.shape(train_labels))


In [0]:
#피클사용 로드
with open('/content/drive/My Drive/2019-ml-finalproject/train_labels.pickle', 'rb') as f:
    train_labels1 = pickle.load(f)
with open('/content/drive/My Drive/2019-ml-finalproject/train_des.pickle', 'rb') as f:
    train_des1 = pickle.load(f)

In [0]:
#SIFT 계산
def computeSIFT(data):
    x = []
    for i in range(0, len(data)):
        sift = cv2.xfeatures2d.SIFT_create()
        img = data[i]
        step_size = 8 # stepsize : 8  range kp 설정.
        kp = [cv2.KeyPoint(x, y, step_size) for x in range(16, img.shape[0]-16, step_size) for y in range(16, img.shape[1]-16, step_size)]
        #keypoints, descriptors = sift.detectAndCompute(img, kp)
        dense_feat = sift.compute(img, kp) 
        x.append(dense_feat[1]) #description 값 계산
        #weak feature --> 나쁜성능
        #keypoints, descriptors = sift.detectAndCompute(img, None)
        #x.append(descriptors)
    return x